# Imports

In [1]:
#!pip install voila
#!pip install numpy
#!pip install matplotlib
#!pip install ipywidgets
#!pip install scipy
#!pip install jupyter_contrib_nbextensions
#!pip install markdown2
#!pip install pandas
#!jupyter contrib nbextension install --sys-prefix
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix --user
#!jupyter serverextension enable voila --sys-prefix

In [2]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import numpy as np
from scipy.integrate import simpson
from numpy import trapz
import matplotlib.pyplot as plt
import markdown2
import pandas as pd
from IPython.display import display
import io
import io
import pandas as pd
import ipywidgets

## Buttons

In [3]:
upload = widgets.FileUpload(multiple=False)
#upload

In [4]:
# with open("input.txt", "w+b") as i:
#     i.write(upload.data[0])

In [5]:
date = widgets.DatePicker(description='Pick a Date')
#date

In [6]:
equip = widgets.ToggleButtons(
            options=['TEM1', 'TEM2']
        )
#equip

In [7]:
sample = widgets.Text(placeholder='Sample number')
#sample

In [8]:
gas = widgets.Dropdown(
    options=['He', 'Air', 'Argon'],
    value='Air',
    description='Carrier gas:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

gas.observe(on_change)

#display(gas)

In [9]:
flow_rate = widgets.ToggleButtons(
            options=['30:10', '15:5']
        )
#flow_rate

In [10]:
values_list = list(range(1, 121))
rec_time = widgets.Dropdown(
    options=values_list,
    value=10,
    description='Recording time, s:',
)
#rec_time

In [11]:
button_send = widgets.Button(
                description='Generate report',
                tooltip='Generate',
                style={'description_width': 'initial'}
            )

output = widgets.Output()
  
def on_button_clicked(event):
    with output:
        clear_output()
#        print("hahaha")
        calculcations()
        generate_plot()
        markdown_content = generate_markdown()
        md = markdown2.markdown(markdown_content)
        display(Markdown(md))
        
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [12]:
text_0 = widgets.HTML(value="<h1>TEM integration!</h1>")
text_00 = widgets.HTML(value="<h2>Upload TEM output file</h2>")
text_1 = widgets.HTML(value="<h2>Please pick an equipment and carrier gas, flow rate and recording time used</h2>")
text_2= widgets.HTML(value="<h2>The experiment was performed on</h2>")
text_3= widgets.HTML(value="<h2>for sample</h2>")


vbox_text = widgets.VBox([text_0, text_00, upload, text_1, equip, gas, flow_rate, rec_time, text_2, date, text_3, sample, vbox_result])

In [13]:
def generate_markdown():
    content = f"""
    # Report

    **Sample Date:** {date.value}  
    **Equipment:** {equip.value}  
    **Gas:** {gas.value}  
    **Sample:** {sample.value}  

    ## Additional Information
    - This is a sample report generated with ipywidgets.
    - The plot above shows tritium signal in time.
    """
    return content

In [14]:
markdown_content = generate_markdown()
md = markdown2.markdown(markdown_content)
#display(Markdown(md))

## Calculations

In [15]:
def activity(x):
    recalc = 3600000
    if equip.value == "TEM1":
       if gas.value == "He":
           cal_const = 85000
       if gas.value == "Air":
           cal_const = 85000
       if gas.value == "Argon":
           cal_const = 85000
    flow = int(flow_rate.value[3:])
    values = x*flow*cal_const/recalc
    return values

In [16]:
def calculcations():
    uploaded_file = upload.value[0]
    pd.read_csv(io.BytesIO(uploaded_file.content))
    df = pd.read_csv(io.BytesIO(uploaded_file.content), sep=" ")
    unique_records = df['Record'].unique()
    record_indices = {}
    for record in unique_records:
        indices = df.index[df['Record'] == record].tolist()
        record_indices[record] = indices
    start = record_indices[record][0]
    backgr = df["Akoinz_cps"].iloc[:start].mean()
    df['Antikoinz_wo_backgr'] = np.where(df['Akoinz_cps'] >= backgr, df['Akoinz_cps'] - backgr, 0)
    df["values"] = activity(df["Antikoinz_wo_backgr"])
    area = trapz(df["values"], dx=rec_time.value)
    print("area, Bq =", area)
    tritium_lambda = 1.78*(10**-9)
    area_atoms = area/tritium_lambda
    print("{:e}".format(area_atoms))

In [23]:
def generate_plot():
    uploaded_file = upload.value[0]
    pd.read_csv(io.BytesIO(uploaded_file.content))
    df = pd.read_csv(io.BytesIO(uploaded_file.content), sep=" ")
    plt.scatter(x=df.Total_sec, y=df.Akoinz_cps)
    plt.xlabel('time, s')
    plt.ylabel('cps')
    plt.savefig('plot.png')
    plt.show()

In [18]:
#print(df.Record.unique())

## Page

In [24]:
page = widgets.HBox([vbox_text])
display(page)

In [25]:
!pip freeze >requirements.txt